In [1]:
import json
import numpy as np
from tqdm import tqdm

In [2]:
from gensim.models import word2vec
from gensim import models
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

In [3]:
pre_data_PATH = './data/'

In [4]:
# with open(pre_data_PATH+'com_product_intro_sentence_articles_test.json') as jsonfile:
#     com_product_intro_sentence = json.load(jsonfile)

In [5]:
with open(pre_data_PATH+'com_product_intro_word_articles_test.json') as jsonfile:
    com_product_intro_word = json.load(jsonfile)
com_product_intro_word

[['椰林 遍佈 島 美托三角洲 離 胡志明市 段 距離 但 許多 遊客 深受 那裡 水鄉澤國 傳統 集市 的 吸引 願意 驅車 前往 ',
  '胡志明市 出發 便利 接送 服務 你 少 交通 的 煩惱 ',
  '我們 搭上 小 船 入坐 馬車 到 濃郁 風情 湄公河 市鎮 一探 吧 ',
  '來到 越南 胡志明市 預訂 印度 支那號 遊船 晚宴 共有 種 套餐 供 選擇 讓 在 欣賞 西貢河 沿岸 夜景 之餘 放鬆 享用 精緻 晚宴 美食 遊船 更 精彩 舞蹈 民謠 音樂 傳統 表演 帶給 胡志明 浪漫 夜晚 特別 體驗 ',
  '胡志明 古芝 地道 私人 小團 日 遊 搭乘 舒適 空調 專車 前往 越南 戰爭 時期 重要 遺跡 全程 專業 中文 英語 導遊 隨行 導覽 帶 仔細 認識 古芝 地道 歷史 背景 適合 人 一同 搭 車 出遊 親身 感受 越戰 時期 緊張 氣氛 ',
  '越南  Viettel  上網 電話卡 前  8 GB  高速 上網 流量 超過 後 降速  64 kbps 效期  15 天 再 加贈 免費 接聽 來電 適用 越南 全 境 河內 胡志明市 重點 觀光區 皆 使用 ， 方便 旅途 中 隨時 卡 上網 ',
  '想 越南 走跳 能 少 最 便宜 最 方便 背包客 必備 交通 方式  Open Bus ',
  '事先 預定 巴士 的 座位 不用 害怕 熱門 時段 不了 車 '],
 ['個 小時 穿越 泰國 古今 了 解 泰國 豐厚 迷人 歷史 文化 ',
  '專業 演員 配合 絢爛 舞台 效果 曼谷暹羅 天使 劇場 你 重新 認識 泰國 ',
  '還可以 開場 前 欣賞 戶外 表演 參觀 民俗 展區 享用 自助 晚餐 品嘗 道地 泰國 美食 ',
  '專業 貼心 中文 導遊 陪伴 下 體驗 傳統 市集 鐵道 共存 奇景 美功 鐵道 市集 暢遊 百 年 歷史 丹嫩莎朵 水 市場 看看 式 特色 小吃 個性 商店 安帕瓦 水上 市場 參觀 獨具 特色 百 年 基督教堂 參訪 泰國 十 靈廟 一 「 樹中廟 」 並 唯美 夜色 中 迎 微風 ， 乘船 觀賞 閃爍 飛舞 螢火蟲 ',
  'Tripadvisor 顆 星 滿分 評價 曼谷泰式 烹飪 教室 乘船 觀賞 昭披 耶河 風光 後 抵達 綠意盎然 私人 香草 花園 

## Load word2vec model

In [6]:
model = models.Word2Vec.load('./word2vec/word2vec.model')

In [7]:
word_embeddings = {}
words = list(model.wv.vocab)
for a in words:
    word_embeddings[a] = model[a]

/home/tommytyc/anaconda3/envs/test_Tempest/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


## TextRank

### Using cosine smilarity between the sentences to build graph

In [8]:
sentence_similarity_martix_list = []
for i in tqdm(range(len(com_product_intro_word))):
#     sentences = com_product_intro_sentence[i]
    sentences_list = com_product_intro_word[i]
    n = len(sentences_list)
    sentence_vectors = []
    for s in sentences_list:
        
        if len(s) != 0:
            v = sum([word_embeddings.get(w, np.zeros((250,))) for w in s.split()])/(len(s.split())+0.001)
        else:
            v = np.zeros((250,))
        if type(v) == float:
            v = np.zeros((250,))
        sentence_vectors.append(v)
    sentence_similarity_martix = np.zeros([n, n])
    for j in range(n):
        for k in range(n):
            if j != k:
                sentence_similarity_martix[j][k] = cosine_similarity(sentence_vectors[j].reshape(1,250), sentence_vectors[k].reshape(1,250))[0,0]
    
    sentence_similarity_martix_list.append(sentence_similarity_martix)
    

100%|██████████| 174/174 [00:09<00:00, 17.99it/s]


In [15]:
sentence_similarity_martix_list

[array([[0.        , 0.57489175, 0.52800149, 0.71079161, 0.70913186,
         0.57281984, 0.58987946, 0.43269259],
        [0.57489175, 0.        , 0.40796417, 0.51185805, 0.59730457,
         0.62983717, 0.63216587, 0.52356695],
        [0.52800149, 0.40796417, 0.        , 0.44282883, 0.44200905,
         0.24480984, 0.32684087, 0.30004885],
        [0.71079161, 0.51185805, 0.44282883, 0.        , 0.7353993 ,
         0.56108058, 0.60066164, 0.49286146],
        [0.70913186, 0.59730457, 0.44200905, 0.7353993 , 0.        ,
         0.69926642, 0.64888964, 0.60600977],
        [0.57281984, 0.62983717, 0.24480984, 0.56108058, 0.69926642,
         0.        , 0.71526794, 0.64056542],
        [0.58987946, 0.63216587, 0.32684087, 0.60066164, 0.64888964,
         0.71526794, 0.        , 0.57157168],
        [0.43269259, 0.52356695, 0.30004885, 0.49286146, 0.60600977,
         0.64056542, 0.57157168, 0.        ]]),
 array([[0.        , 0.43564045, 0.53987149, 0.64011201, 0.57389431,
         

### Sentence Ranking

In [9]:
ranked_sentence_list = []
for i in tqdm(range(len(sentence_similarity_martix_list))):
#     sentences = com_product_intro_sentence[i]
    sentences_list = com_product_intro_word[i]
    sentence_similarity_martix = sentence_similarity_martix_list[i]
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
#     try:
    scores = nx.pagerank_numpy(sentence_similarity_graph)
    
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences_list)), reverse=True)
    ranked_sentence_list.append(ranked_sentence)
      

100%|██████████| 174/174 [00:00<00:00, 901.05it/s] 


In [16]:
ranked_sentence_list

[[(0.1411680042248546,
   '胡志明 古芝 地道 私人 小團 日 遊 搭乘 舒適 空調 專車 前往 越南 戰爭 時期 重要 遺跡 全程 專業 中文 英語 導遊 隨行 導覽 帶 仔細 認識 古芝 地道 歷史 背景 適合 人 一同 搭 車 出遊 親身 感受 越戰 時期 緊張 氣氛 '),
  (0.13231637114014297,
   '椰林 遍佈 島 美托三角洲 離 胡志明市 段 距離 但 許多 遊客 深受 那裡 水鄉澤國 傳統 集市 的 吸引 願意 驅車 前往 '),
  (0.13114537735866827, '想 越南 走跳 能 少 最 便宜 最 方便 背包客 必備 交通 方式  Open Bus '),
  (0.13043507570364082,
   '來到 越南 胡志明市 預訂 印度 支那號 遊船 晚宴 共有 種 套餐 供 選擇 讓 在 欣賞 西貢河 沿岸 夜景 之餘 放鬆 享用 精緻 晚宴 美食 遊船 更 精彩 舞蹈 民謠 音樂 傳統 表演 帶給 胡志明 浪漫 夜晚 特別 體驗 '),
  (0.13042457357545117,
   '越南  Viettel  上網 電話卡 前  8 GB  高速 上網 流量 超過 後 降速  64 kbps 效期  15 天 再 加贈 免費 接聽 來電 適用 越南 全 境 河內 胡志明市 重點 觀光區 皆 使用 ， 方便 旅途 中 隨時 卡 上網 '),
  (0.12547632318333832, '胡志明市 出發 便利 接送 服務 你 少 交通 的 煩惱 '),
  (0.11664941790237207, '事先 預定 巴士 的 座位 不用 害怕 熱門 時段 不了 車 '),
  (0.09238485691153184, '我們 搭上 小 船 入坐 馬車 到 濃郁 風情 湄公河 市鎮 一探 吧 ')],
 [(0.090640532523516,
   '專業 貼心 中文 導遊 陪伴 下 體驗 傳統 市集 鐵道 共存 奇景 美功 鐵道 市集 暢遊 百 年 歷史 丹嫩莎朵 水 市場 看看 式 特色 小吃 個性 商店 安帕瓦 水上 市場 參觀 獨具 特色 百 年 基督教堂 參訪 泰國 十 靈廟 一 「 樹中廟 」 並 唯美 夜色 中 迎 微風 ， 乘船 觀賞 閃爍 飛

### Sentence Selection

In [10]:
from functools import reduce

In [11]:
num = 6 # choose top n sentences
sentence_select_list = []
for s in ranked_sentence_list:
    l = min(len(s),num)
    sentence_select = [y for (x,y) in s[0:l]]
#     sentence_select = s[0:l]
    
    sentence_select = reduce(lambda x, y:str(x)+str(y),sentence_select)
    sentence_select_list.append(sentence_select)
    

In [12]:
result_PATH = './data/result/'

In [13]:
with open(result_PATH +'textrank.json', 'w') as outfile:
    json.dump(sentence_select_list, outfile)

In [14]:
sentence_select_list

['胡志明 古芝 地道 私人 小團 日 遊 搭乘 舒適 空調 專車 前往 越南 戰爭 時期 重要 遺跡 全程 專業 中文 英語 導遊 隨行 導覽 帶 仔細 認識 古芝 地道 歷史 背景 適合 人 一同 搭 車 出遊 親身 感受 越戰 時期 緊張 氣氛 椰林 遍佈 島 美托三角洲 離 胡志明市 段 距離 但 許多 遊客 深受 那裡 水鄉澤國 傳統 集市 的 吸引 願意 驅車 前往 想 越南 走跳 能 少 最 便宜 最 方便 背包客 必備 交通 方式  Open Bus 來到 越南 胡志明市 預訂 印度 支那號 遊船 晚宴 共有 種 套餐 供 選擇 讓 在 欣賞 西貢河 沿岸 夜景 之餘 放鬆 享用 精緻 晚宴 美食 遊船 更 精彩 舞蹈 民謠 音樂 傳統 表演 帶給 胡志明 浪漫 夜晚 特別 體驗 越南  Viettel  上網 電話卡 前  8 GB  高速 上網 流量 超過 後 降速  64 kbps 效期  15 天 再 加贈 免費 接聽 來電 適用 越南 全 境 河內 胡志明市 重點 觀光區 皆 使用 ， 方便 旅途 中 隨時 卡 上網 胡志明市 出發 便利 接送 服務 你 少 交通 的 煩惱 ',
 '專業 貼心 中文 導遊 陪伴 下 體驗 傳統 市集 鐵道 共存 奇景 美功 鐵道 市集 暢遊 百 年 歷史 丹嫩莎朵 水 市場 看看 式 特色 小吃 個性 商店 安帕瓦 水上 市場 參觀 獨具 特色 百 年 基督教堂 參訪 泰國 十 靈廟 一 「 樹中廟 」 並 唯美 夜色 中 迎 微風 ， 乘船 觀賞 閃爍 飛舞 螢火蟲 還可以 開場 前 欣賞 戶外 表演 參觀 民俗 展區 享用 自助 晚餐 品嘗 道地 泰國 美食 Tripadvisor 顆 星 滿分 評價 曼谷泰式 烹飪 教室 乘船 觀賞 昭披 耶河 風光 後 抵達 綠意盎然 私人 香草 花園 學習 香草 知識 由 經驗 豐富 Amita 奶奶 傳授 道 經典 泰國 料理 秘訣 大城 旅遊 走馬看花 不 是 生硬 歷史 教材 透過 中文 專業 導遊 深入淺出 介紹 以 幽默 方式 認識 座 擁有 豐富 歷史 文化 城市 大城 看遍 的 繁華 興衰 個 小時 穿越 泰國 古今 了 解 泰國 豐厚 迷人 歷史 文化 2020 期間 限定 再 贈 4GB 20 分鐘 通話卡 8 天 GB 高流量 上網 通話 輕